<a href="https://colab.research.google.com/github/juanfrans-courses/BuildPeaceWorkshop/blob/master/threads-storytelling-with-maps-and-data/04_Public_Data/Session_3_Thinking_About_Data_Part_3_Census_Cont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Census Bureau
#### The Decennial Census and the American Community Survey

*At first glance data are apparently before the fact, they are the starting point for what we know, who we are, and how we communicate. This shared sense of starting with data often leads to an unnoticed assumption that data are transparent, that information is self-evident, the fundamental stuff of truth itself.* - Jackson Gitelman

### Getting Set Up

Before we install our packages and start to work with census data, we need to register for a Census API Key so that we can remove the majority of restrictions when it comes to accessing and downloading Census Data.

To do so, go to [api.census.gov/data/key_signup.html](https://api.census.gov/data/key_signup.html) and provide your basic access information (name and organization).

Store your API key in a place familiar to you, like a password manager.

In [ ]:
!pip install geopandas
!pip install cenpy
!pip install jenkspy
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
import cenpy as cen
import cenpy.tiger as tiger
import jenkspy
from getpass import getpass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 11.2 MB/s 
     |████████████████████████████████| 16.7 MB 52.9 MB/s 
     |████████████████████████████████| 6.3 MB 17.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 32.4 MB/s 
     |████████████████████████████████| 2.4 MB 69.7 MB/s 
  Created wheel for cenpy: filename=cenpy-1.0.1-py3-none-any.whl size=30061 sha256=9d34947e946c76d8fe130d47aec84b330b30873f99fb735bb4cf988f26e19944
  Stored in directory: /root/.cache/pip/wheels/7f/5c/d8/459cb49ab185291b4906ac42dde99ee6b264aa279a721caa60
Successfully built cenpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 495 kB 14.1 MB/s 


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
CENSUS_API_KEY = getpass('Enter your Census API Key: ')

Enter your Census API Key: ··········


In [ ]:
!pip install altair-latimes
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ThemeRegistry.enable('latimes')

### Exploring Census Data

To access and download Census data, we are going to use the library Cenpy. I chose this over other Census libraries because it is simple to use, and for certain API calls, doesn't require importing geography separately. To begin, let's simply see what is available through the API.



In [ ]:
availableTables = cen.explorer.available()
print(f"The API offers access to {len(availableTables)} datasets")

The API offers access to 1285 datasets


In [ ]:
availableTables.head(10)

,c_isTimeseries,temporal,spatial,c_isAggregate,publisher,references,programCode,modified,license,keyword,...,description,bureauCode,accessLevel,title,c_isAvailable,c_isCube,c_isMicrodata,c_documentationLink,c_dataset,vintage
ABSCB2017,NaN,NaN,NaN,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2020-04-30 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscb,)",2017.0
ABSCB2018,NaN,NaN,NaN,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2020-10-26 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscb,)",2018.0
ABSCB2019,NaN,NaN,NaN,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2021-08-17 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscb,)",2019.0
ABSCB2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2022-08-03 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscb,)",2020.0
ABSCBO2017,NaN,2017/2017,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2020-04-30 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscbo,)",2017.0
ABSCBO2018,NaN,2018/2018,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2020-10-26 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscbo,)",2018.0
ABSCBO2019,NaN,2019/2019,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2021-08-17 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscbo,)",2019.0
ABSCBO2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2022-08-03 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscbo,)",2020.0
ABSCS2017,NaN,NaN,NaN,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2020-04-30 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Annual Business Survey: Company Summary: 2017,True,NaN,NaN,https://www.census.gov/developer/,"(abscs,)",2017.0
ABSCS2018,NaN,NaN,NaN,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2020-10-26 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscs,)",2018.0


This can be a little overwhelming to look through. Thankfully, Cenpy build in functionality to simply look through ACS products.

As a reminder, the American Community Survey data comes in 1-year, 3-year (discontinued), and 5-year surveys. Due to the sample size for each of these surveys, we prioritize the 5-year over the 3-year, and the 3-year (when looking at historic data) over the 1-year. Generally, you will be balancing accuracy and the margin of error, with the relevancy of the data and how recent the survey was conducted. This, along with the breadth of questions asked, is why we turn to the ACS over the Decennial Census data, despite the Census being a full enumeration.

Let's begin by filtering down to ACS data.


In [ ]:
acs = cen.products.ACS()

Next, we can look through what tables are available to us within the ACS product.



In [ ]:
acs.tables

,description,columns
table_name,,
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B01003,TOTAL POPULATION,[B01003_001E]
B02001,RACE,"[B02001_001E, B02001_002E, B02001_003E, B02001..."
B02008,WHITE ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02008_001E]
...,...,...
C27014,PUBLIC HEALTH INSURANCE BY WORK EXPERIENCE,"[C27014_001E, C27014_002E, C27014_003E, C27014..."
C27016,HEALTH INSURANCE COVERAGE STATUS BY RATIO OF I...,"[C27016_001E, C27016_002E, C27016_003E, C27016..."
C27017,PRIVATE HEALTH INSURANCE BY RATIO OF INCOME TO...,"[C27017_001E, C27017_002E, C27017_003E, C27017..."


Now, using simple filters, we can query the ACS products efficiently.

In [ ]:
acs.filter_tables('RACE', by='description')

,description,columns
table_name,,
B02001,RACE,"[B02001_001E, B02001_002E, B02001_003E, B02001..."
B02008,WHITE ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02008_001E]
B02009,BLACK OR AFRICAN AMERICAN ALONE OR IN COMBINAT...,[B02009_001E]
B02010,AMERICAN INDIAN AND ALASKA NATIVE ALONE OR IN ...,[B02010_001E]
B02011,ASIAN ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02011_001E]
B02012,NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALO...,[B02012_001E]
B02013,SOME OTHER RACE ALONE OR IN COMBINATION WITH O...,[B02013_001E]
B03002,HISPANIC OR LATINO ORIGIN BY RACE,"[B03002_001E, B03002_002E, B03002_003E, B03002..."
B25006,RACE OF HOUSEHOLDER,"[B25006_001E, B25006_002E, B25006_003E, B25006..."


In [ ]:
acs.filter_tables('HISPANIC', by='description')

,description,columns
table_name,,
B03001,HISPANIC OR LATINO ORIGIN BY SPECIFIC ORIGIN,"[B03001_001E, B03001_002E, B03001_003E, B03001..."
B03002,HISPANIC OR LATINO ORIGIN BY RACE,"[B03002_001E, B03002_002E, B03002_003E, B03002..."
B03003,HISPANIC OR LATINO ORIGIN,"[B03003_001E, B03003_002E, B03003_003E]"
B16006,LANGUAGE SPOKEN AT HOME BY ABILITY TO SPEAK EN...,"[B16006_001E, B16006_002E, B16006_003E, B16006..."
B98013,TOTAL POPULATION COVERAGE RATE BY WEIGHTING RA...,"[B98013_001E, B98013_002E, B98013_003E, B98013..."
B99031,ALLOCATION OF HISPANIC OR LATINO ORIGIN,"[B99031_001E, B99031_002E, B99031_003E]"


You can also filter by the table name, which is easy to find using Census Reporter.

In [ ]:
acs.filter_tables('B030', by='table_name')

,description,columns
table_name,,
B03001,HISPANIC OR LATINO ORIGIN BY SPECIFIC ORIGIN,"[B03001_001E, B03001_002E, B03001_003E, B03001..."
B03002,HISPANIC OR LATINO ORIGIN BY RACE,"[B03002_001E, B03002_002E, B03002_003E, B03002..."
B03003,HISPANIC OR LATINO ORIGIN,"[B03003_001E, B03003_002E, B03003_003E]"


Once you know what table you're interested in viewing, let's look into the variables contained within the table. For this exercise, we are going to make a few maps using census data:


1.   Map of Median Household Income (by County) for Minnesota
2.   Map of Gini Index (by County) for the US
2.   Map of Rent as a Percent of Income (by Tract) for cities and states

On Census reporter, I can quickly identify that the table I want to use for the first map is B19013 and the table I want to use for the second is B25071.

Using the filter function below, we can quickly identify the columns we want to include.

In [ ]:
acs.filter_variables('B19013')

,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
B19013E_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013E,0,NaN,NaN,"B19013E_001EA,B19013E_001M,B19013E_001MA",NaN
B19013D_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013D,0,NaN,NaN,"B19013D_001EA,B19013D_001M,B19013D_001MA",NaN
B19013I_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013I,0,NaN,NaN,"B19013I_001EA,B19013I_001M,B19013I_001MA",NaN
B19013A_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013A,0,NaN,NaN,"B19013A_001EA,B19013A_001M,B19013A_001MA",NaN
B19013F_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013F,0,NaN,NaN,"B19013F_001EA,B19013F_001M,B19013F_001MA",NaN
B19013G_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013G,0,NaN,NaN,"B19013G_001EA,B19013G_001M,B19013G_001MA",NaN
B19013B_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013B,0,NaN,NaN,"B19013B_001EA,B19013B_001M,B19013B_001MA",NaN
B19013C_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013C,0,NaN,NaN,"B19013C_001EA,B19013C_001M,B19013C_001MA",NaN
B19013_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013,0,NaN,NaN,"B19013_001EA,B19013_001M,B19013_001MA",NaN
B19013H_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,int,B19013H,0,NaN,NaN,"B19013H_001EA,B19013H_001M,B19013H_001MA",NaN


In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
acs.filter_variables('B19013')

,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
B19013E_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE HOUSEHOLDER),int,B19013E,0,NaN,NaN,"B19013E_001EA,B19013E_001M,B19013E_001MA",NaN
B19013D_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (ASIAN ALONE HOUSEHOLDER),int,B19013D,0,NaN,NaN,"B19013D_001EA,B19013D_001M,B19013D_001MA",NaN
B19013I_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (HISPANIC OR LATINO HOUSEHOLDER),int,B19013I,0,NaN,NaN,"B19013I_001EA,B19013I_001M,B19013I_001MA",NaN
B19013A_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (WHITE ALONE HOUSEHOLDER),int,B19013A,0,NaN,NaN,"B19013A_001EA,B19013A_001M,B19013A_001MA",NaN
B19013F_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (SOME OTHER RACE ALONE HOUSEHOLDER),int,B19013F,0,NaN,NaN,"B19013F_001EA,B19013F_001M,B19013F_001MA",NaN
B19013G_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (TWO OR MORE RACES HOUSEHOLDER),int,B19013G,0,NaN,NaN,"B19013G_001EA,B19013G_001M,B19013G_001MA",NaN
B19013B_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER),int,B19013B,0,NaN,NaN,"B19013B_001EA,B19013B_001M,B19013B_001MA",NaN
B19013C_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (AMERICAN INDIAN AND ALASKA NATIVE ALONE HOUSEHOLDER),int,B19013C,0,NaN,NaN,"B19013C_001EA,B19013C_001M,B19013C_001MA",NaN
B19013_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS),int,B19013,0,NaN,NaN,"B19013_001EA,B19013_001M,B19013_001MA",NaN
B19013H_001E,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),"MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (WHITE ALONE, NOT HISPANIC OR LATINO HOUSEHOLDER)",int,B19013H,0,NaN,NaN,"B19013H_001EA,B19013H_001M,B19013H_001MA",NaN


Here, I see that I want to pull in the column B19013_001E from this table

*MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS)*

In [ ]:
acs.filter_variables('B25071')

,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
B25071_001E,Estimate!!Median gross rent as a percentage of household income,MEDIAN GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS (DOLLARS),float,B25071,0,NaN,NaN,"B25071_001EA,B25071_001M,B25071_001MA",NaN


And for the table B25071, this is much easier to identify as its the only column. (B25071_001E)

*MEDIAN GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS (DOLLARS)*

In [ ]:
# Let's reset our pandas display
pd.reset_option('display.max_colwidth')

## Downloading Census Data

#### Map 1

##### Downloading Census Data

As a reminder, for the first map, we will be highlighting Median Household Income (by County) for Minnesota

To do this, we will be using Table B19013 / Column B19013_001E

In [ ]:
availableTables.loc[availableTables['vintage'] == 2020.0]

,c_isTimeseries,temporal,spatial,c_isAggregate,publisher,references,programCode,modified,license,keyword,...,description,bureauCode,accessLevel,title,c_isAvailable,c_isCube,c_isMicrodata,c_documentationLink,c_dataset,vintage
ABSCB2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2022-08-03 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscb,)",2020.0
ABSCBO2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2022-08-03 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscbo,)",2020.0
ABSCS2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2022-08-03 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(abscs,)",2020.0
ABSMCB2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:007,2022-08-03 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Annual Business Survey (ABS) provides info...,006:07,public,Economic Surveys: Annual Business Survey: Annu...,True,NaN,NaN,https://www.census.gov/developer/,"(absmcb,)",2020.0
ACSCP5Y2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:004,2021-07-13 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The American Community Survey (ACS) is an ongo...,006:07,public,American Community Survey: 5-Year Estimates: C...,True,True,NaN,https://www.census.gov/developer/,"(acs, acs5, cprofile)",2020.0
ACSDP5Y2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:004,2021-07-13 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The American Community Survey (ACS) is an ongo...,006:07,public,American Community Survey: 5-Year Estimates: D...,True,True,NaN,https://www.census.gov/developer/,"(acs, acs5, profile)",2020.0
ACSDT5Y2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:004,2021-07-13 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The American Community Survey (ACS) is an ongo...,006:07,public,American Community Survey: 5-Year Estimates: D...,True,True,NaN,https://www.census.gov/developer/,"(acs, acs5)",2020.0
ACSPUMS5Y2020,NaN,NaN,NaN,NaN,U.S. Census Bureau,https://www.census.gov/developers/,006:004,2022-01-31 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The American Community Survey (ACS) Public Use...,006:07,public,2020 American Community Survey: 5-Year Estimat...,True,True,True,https://www.census.gov/developer/,"(acs, acs5, pums)",2020.0
ACSPUMS5YPR2020,NaN,NaN,NaN,NaN,U.S. Census Bureau,https://www.census.gov/developers/,006:004,2022-01-31 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The Public Use Microdata Sample (PUMS) for Pue...,006:07,public,2020 American Community Survey: 5-Year Estimat...,True,True,True,https://www.census.gov/developer/,"(acs, acs5, pumspr)",2020.0
ACSST5Y2020,NaN,2020/2020,US,True,U.S. Census Bureau,https://www.census.gov/developers/,006:004,2021-07-13 00:00:00.0,https://creativecommons.org/publicdomain/zero/...,"(census,)",...,The American Community Survey (ACS) is an ongo...,006:07,public,American Community Survey: 5-Year Estimates: S...,True,True,NaN,https://www.census.gov/developer/,"(acs, acs5, subject)",2020.0


Here we can see the various tables collected by the American Community Survey. Because we are solely interested in basic populations statistics, we will be pulling from the `ACSDT5Y2019` (Detailed Tables).

In [ ]:
# Here, we are setting a variable (con) for making a call to this specific table.
con = cen.remote.APIConnection('ACSDT5Y2020',apikey=CENSUS_API_KEY)

# Next we will look at the variables available to us in this table.
variables = con.variables

# We then want to identify the columns we want to map. Here, I will include Median Household Income, and total population (which should be included by default)
columns = [
    'B01003_001', # Total Population
    'B19013_001', # Median Household Income
]

# Notice the E was absent from the columns identified. The E identifies the column as an estimate. But we also want to include margins of error with our estimate. Hence the code below.
columns_E = [i+'E' for i in columns]
columns_M = [i+'M' for i in columns]

# Define the units of geography, as well as a filter for what we are looking for.
# State codes can be found at https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
# You can also use the Missouri Census Data Center, which has great resources for lookups - https://mcdc.missouri.edu/
# Geographies: zip code tabulation area, school district (elementary), school district (secondary), combined statistical area, congressional district, county, tract, block group
# Filters on the geography: filter to tracts within state and county = {'state':'12', 'county':'073'}
g_unit = 'county'
g_filter = {'state':'27'}

# Make an API call for our tables/columns at the county level for the state of Minnesota and save to DF
mnIncome = con.query(columns_E + columns_M, geo_unit=g_unit, geo_filter=g_filter)

In [ ]:
mnIncome.head()

,B01003_001E,B19013_001E,B01003_001M,B19013_001M,state,county
0,353775,84379,-555555555,1323,27,003
1,34227,60508,-555555555,2064,27,005
2,46784,50525,-555555555,2744,27,007
3,40476,60564,-555555555,2877,27,009
4,4974,55909,-555555555,2966,27,011


##### Merging Census Data

As referenced above, Census survey data comes separate from Census geography data. To get this data in a format that we can map, we need to create a geodataframe that incorporates both the geography as well as the survey estimates. To do so, we will join the two dataframes on a common value -- the `GEOID`.

Let's start with the County Data.

In [ ]:
usCounties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_5m.zip')

In [ ]:
usCounties.head()

In [ ]:
mnGeo = usCounties[usCounties['STATEFP'] == '27']

In [ ]:
mnGeo.head(1)

Above, I can see that the counties geography data has a GEOID, but the counties estimates data dies not. However, it does have the two elements that comprise a matching `GEOID` field -- `state` and `county`. So let's create a new column that is simply the addition of these two columns.

In [ ]:
mnIncome['GEOID'] = mnIncome['state'] + mnIncome['county']

In [ ]:
print(mnGeo.GEOID)
print(mnIncome.GEOID)

These look to match, and we also can see that both dataframes have 87 rows. Now we can merge on this column.

In [ ]:
mnIncome = mnGeo.merge(mnIncome, on='GEOID')

In [ ]:
mnIncome.head()

In [ ]:
mnIncomeMap = alt.Chart(mnIncome).mark_geoshape().encode(
    color=alt.Color('B19013_001E:Q', legend=alt.Legend(title='Median Household Income (Dollars)', format='$,.2r')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('B01003_001E',title='Population', format=',.2r'),
      alt.Tooltip('B19013_001E',title='Median Household Income', format='$,.2r')]
).properties(
    width=700,
    height=700,
    title='Median Household Income (MN)'
)

mnIncomeMap

#### Map 2

As a reminder, for the first map, we will be highlighting Gini index (by County) for Minnesota

To do this, we will be using Table 19083 / Column B19083_001E.

We will use very similar variables to before, but for our geography filter, we will use a wildcard as opposed to a specific state.

In [ ]:
# Here, we are setting a variable (con) for making a call to this specific table. We will setup two data pulls, one for 2019 and one for 2014.
con = cen.remote.APIConnection('ACSDT5Y2020',apikey=CENSUS_API_KEY)

# Next we will look at the variables available to us in this table.
variables = con.variables

# We then want to identify the columns we want to map. Here, I will include Median Household Income, and total population (which should be included by default)
columns = [
    'B01003_001', # Total Population
    'B19083_001', # Gini Index
]

# Notice the E was absent from the columns identified. The E identifies the column as an estimate. But we also want to include margins of error with our estimate. Hence the code below.
columns_E = [i+'E' for i in columns]
columns_M = [i+'M' for i in columns]

# Define the units of geography, as well as a filter for what we are looking for.
# State codes can be found at https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
# You can also use the Missouri Census Data Center, which has great resources for lookups - https://mcdc.missouri.edu/
g_unit = 'county'
g_filter = {'state':'*'}

# Make an API call for our tables/columns at the county level for the state of Georgia and save to DF
usGini = con.query(columns_E + columns_M, geo_unit=g_unit, geo_filter=g_filter)

In [ ]:
usGini.head()

In [ ]:
usGini['GEOID'] = usGini['state'] + usGini['county']
usGini.head()

In [ ]:
usGini = usCounties.merge(usGini, on='GEOID')

In [ ]:
giniMap = alt.Chart(usGini).mark_geoshape().encode(
    color=alt.Color('B19083_001E:Q',
      legend=alt.Legend(title='Gini Index'),
      scale=alt.Scale()),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('B19083_001E',title='Gini Index')]
).project(
    type='albersUsa'
).properties(
    title='US Gini Index (by County)',
    width=700,
    height=450
).configure_view(
    strokeWidth=0
)

giniMap

In [ ]:
usGini['B19083_001E'] = usGini['B19083_001E'].astype(float)

In [ ]:
breaks = jenkspy.jenks_breaks(usGini['B19083_001E'], nb_class=5)
breaks

In [ ]:
usGini['naturalBreaks'] = pd.cut(usGini['B19083_001E'], breaks, labels=['.3023 - .411', '.412 - .443', '.444 - .4753', '.4753-.5160', '.5161 - .707'])

In [ ]:
usStates = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_state_5m.zip')

In [ ]:
usStateMap = alt.Chart(usStates).mark_geoshape(
    fill=None,
    stroke='white',
    strokeWidth=0.5
)

giniMap = alt.Chart(usGini).mark_geoshape().encode(
    color=alt.Color('naturalBreaks:O',
      legend=alt.Legend(title='Gini Index'),
      scale=alt.Scale()),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('B19083_001E',title='Gini Index')]
).project(
    type='albersUsa'
).properties(
    title='US Gini Index (by County)',
    width=700,
    height=450
)

giniMap + usStateMap

#### Map 3

For our third map, we will be highlighting Rent as a Percent of Income (by Tract) for Pittsburgh, Chicago, and the state of Montana. For these maps, we will be using the `.from_` function.

For this map, we will use Table B25071 / Column B25071_001E

In [ ]:
rentIncomeMN = cen.products.ACS(2019).from_msa('Pittsburgh, PA', level='tract', variables=['B25071_001E', 'B19013_001E'])
#rentIncomeChicago = cen.products.ACS(2019).from_place('Chicago, IL', level='tract', variables=['B25071_001E', 'B19013_001E'])
#rentIncomeMontana = cen.products.ACS(2019).from_state('Montana', level='tract', variables=['B25071_001E', 'B19013_001E'])

This is where Cenpy shines. Grabbing both data, and geography, all through a simple command. In the last example, I am downloading a state geography file, and columns from two data tables, at the census tract level, all in a single line.

Now let's make our first map.

In [ ]:
alt.Chart(rentIncomePittsburgh).mark_geoshape()

So what's happening here? This is a very common error but thankfully a very easy fix! Whenever you see a chart that looks like this, it means you are trying to map something without a CRS, or with the wrong CRS. So let's apply our standard WGS84 CRS to all three geodataframes.

In [ ]:
rentIncomePittsburgh = rentIncomePittsburgh.to_crs({'init': 'epsg:4326'})
rentIncomeChicago = rentIncomeChicago.to_crs({'init': 'epsg:4326'})
rentIncomeMontana = rentIncomeMontana.to_crs({'init': 'epsg:4326'})

In [ ]:
rentIncomePittsburgh.head()

In [ ]:
rentIncomeChicago.head()

In [ ]:
rentIncomeMontana.head()

In [ ]:
rentIncomePittsburghMap = alt.Chart(rentIncomePittsburgh).mark_geoshape().encode(
    color=alt.Color('B25071_001E:Q', legend=alt.Legend(title='Rent as a Percent of Income')),
    tooltip=[alt.Tooltip('NAME',title='Census Tract'),
      alt.Tooltip('B25071_001E',title='Rent as % of Income')]
).properties(
    width=700,
    height=700,
    title='Rent as a % of Income (Pittsburgh Metropolitan Area)'
).project(
    type='conicConformal',
    parallels=[39 + 56 / 60, 40 + 58 / 60],
    rotate=[77 + 45 / 60, 0]
)

rentIncomePittsburghMap

In [ ]:
incomeChicagoMap = alt.Chart(rentIncomeChicago).mark_geoshape().encode(
    color=alt.Color('B19013_001E:Q', legend=alt.Legend(title='Median Household Income')),
    tooltip=[alt.Tooltip('B19013_001E',title='Median Household Income')]
).properties(
    width=700,
    height=700,
    title='Median Household Income (Chicago)'
).project(
    type='transverseMercator',
    rotate=[90 + 10 / 60, -36 - 40 / 60]
)

rentIncomeChicagoMap = alt.Chart(rentIncomeChicago).mark_geoshape().encode(
    color=alt.Color('B25071_001E:Q', legend=alt.Legend(title='Rent as a Percent of Income')),
    tooltip=[alt.Tooltip('B25071_001E',title='Rent as % of Income')]
).properties(
    width=700,
    height=700,
    title='Rent as a % of Income (Chicago)'
).project(
    type='transverseMercator',
    rotate=[90 + 10 / 60, -36 - 40 / 60]
)

rentIncomeChicagoMap

In [ ]:
incomeChicagoMap

In [ ]:
rentIncomeMontanaMap = alt.Chart(rentIncomeMontana).mark_geoshape().encode(
    color=alt.Color('B25071_001E:Q', legend=alt.Legend(title='Rent as a Percent of Income')),
    tooltip=[alt.Tooltip('NAME',title='Census Tract'),
      alt.Tooltip('B25071_001E',title='Rent as % of Income')]
).properties(
    width=700,
    height=700,
    title='Rent as a % of Income (Montana)'
).project(
    type='conicConformal',
    parallels=[45, 49],
    rotate=[109 + 30 / 60, 0]
)

rentIncomeMontanaMap